# Preprocess the bank marketing dataset

- Read the original [bank-additional-full.csv](https://archive.ics.uci.edu/ml/datasets/bank+marketing) dataset,
which consists of 20 features and a binary classification label.

- Preprocess steps
    - remove samples that have unknown values
    - encode categorical features
    - save as bank_additional_full_filtered.csv
    - down-sampling negative samples and save as bank_additional_full_filtered_balanced.csv

- Preprocess for demo split neural network
    - split bank_additional_full_filtered_balanced.csv into three parts
    - first part as bank_additional_full_filtered_balanced_active.csv, which consists of
    the first 10 features and the labels
    - second part as bank_additional_full_filtered_balanced_passive.csv, which consists of
    the last 10 features and the labels
    - third party as bank_additional_full_filtered_balanced_server.csv, which consists of
    the labels. To use the pytorch data loader, duplicate the labels twice

In [15]:
import pandas as pd
import numpy as np
import torch
import random
from torch.utils.data import Dataset

In [16]:
df = pd.read_csv('dataset/bank-additional-full.csv', sep=';')
print(df.describe())
print(df.dtypes)

               age      duration      campaign         pdays      previous  \
count  41188.00000  41188.000000  41188.000000  41188.000000  41188.000000   
mean      40.02406    258.285010      2.567593    962.475454      0.172963   
std       10.42125    259.279249      2.770014    186.910907      0.494901   
min       17.00000      0.000000      1.000000      0.000000      0.000000   
25%       32.00000    102.000000      1.000000    999.000000      0.000000   
50%       38.00000    180.000000      2.000000    999.000000      0.000000   
75%       47.00000    319.000000      3.000000    999.000000      0.000000   
max       98.00000   4918.000000     56.000000    999.000000      7.000000   

       emp.var.rate  cons.price.idx  cons.conf.idx     euribor3m   nr.employed  
count  41188.000000    41188.000000   41188.000000  41188.000000  41188.000000  
mean       0.081886       93.575664     -40.502600      3.621291   5167.035911  
std        1.570960        0.578840       4.628198    

In [17]:
bank_np = df.to_numpy()
print(bank_np[0])
print(bank_np.shape)

[56 'housemaid' 'married' 'basic.4y' 'no' 'no' 'no' 'telephone' 'may'
 'mon' 261 1 999 0 'nonexistent' 1.1 93.994 -36.4 4.857 5191.0 'no']
(41188, 21)


### Remove samples that have unknown values

In [18]:
def unknown_count(arr): 
    count = 0
    arr_idx = []  
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            if arr[i][j] == "unknown":
                arr_idx.append(i)
                count += 1
                break
    return count, arr_idx
            
bank_count, bank_arr_idx = unknown_count(bank_np)
print("Before filtering, unknown value:", bank_count)
print("Dataset shape:", bank_np.shape)

bank_np = np.delete(bank_np, bank_arr_idx, axis=0)
bank_count, _ = unknown_count(bank_np)
print("After filtering, unknown value:", bank_count)
print("Dataset shape:", bank_np.shape)

Before filtering, unknown value: 10700
Dataset shape: (41188, 21)
After filtering, unknown value: 0
Dataset shape: (30488, 21)


### Encode categorical features

In [19]:
print("Before coding:\n", bank_np.shape, bank_np[0])
column_categorical_idx = [1 if 'str' in str(type(ele)) else 0 for ele in bank_np[0, :]]
for cln_idx in range(len(column_categorical_idx)):
    if column_categorical_idx[cln_idx]:
        _, indices = np.unique(bank_np[:, cln_idx], return_inverse=True)
        bank_np[:, cln_idx] = indices
print("After coding:\n", bank_np.shape, bank_np[0])

# save the encoded dataset into csv file
np.savetxt("dataset/bank_additional_full_filtered.csv", bank_np, delimiter=",")

Before coding:
 (30488, 21) [56 'housemaid' 'married' 'basic.4y' 'no' 'no' 'no' 'telephone' 'may'
 'mon' 261 1 999 0 'nonexistent' 1.1 93.994 -36.4 4.857 5191.0 'no']
After coding:
 (30488, 21) [56 3 1 0 0 0 0 1 6 1 261 1 999 0 1 1.1 93.994 -36.4 4.857 5191.0 0]


### Down-sampling negative data samples

Since the bank marketing dataset is highly imbalance, down-sampling to
ensure it is balanced.

In [22]:
# bank dataset is highly imbalanced, down-sample the negative samples
bank_np_labels = bank_np[:,-1]
bank_np_samples = bank_np[:,:-1]
balance_num = int(sum(bank_np_labels) * 2)
bank_balance_labels = np.zeros(balance_num)
bank_balance_samples = np.zeros((balance_num, bank_np_samples.shape[1]))
balance_index = 0
negative_counter = 0
idxes = list(range(len(bank_np_labels)))
random.shuffle(idxes)
for i in idxes:
    if bank_np_labels[i] == 0 and negative_counter < balance_num / 2:
        negative_counter += 1
        bank_balance_labels[balance_index] = bank_np_labels[i]
        bank_balance_samples[balance_index] = bank_np_samples[i]
        balance_index += 1
    elif bank_np_labels[i] == 1:
        bank_balance_labels[balance_index] = bank_np_labels[i]
        bank_balance_samples[balance_index] = bank_np_samples[i]
        balance_index += 1
    if balance_index >= balance_num:
        break

In [24]:
# save the down-sampled bank dataset
bank_np_balanced = bank_balance_samples
bank_np_label = bank_balance_labels.reshape((-1, 1))
bank_np_balanced = np.append(bank_np_balanced, bank_np_label, axis=1)
print("bank_np_balanced[:2,:] = ", bank_np_balanced[:2,:])
np.savetxt("dataset/bank_additional_full_filtered_balanced.csv", bank_np_balanced, delimiter=",")

bank_np_balanced[:2,:] =  [[ 3.9000e+01  2.0000e+00  1.0000e+00  1.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  6.0000e+00  2.0000e+00  1.2100e+02  1.0000e+00
   9.9900e+02  0.0000e+00  1.0000e+00 -1.8000e+00  9.2893e+01 -4.6200e+01
   1.3270e+00  5.0991e+03  0.0000e+00]
 [ 2.7000e+01  0.0000e+00  2.0000e+00  5.0000e+00  0.0000e+00  0.0000e+00
   1.0000e+00  0.0000e+00  3.0000e+00  3.0000e+00  5.1800e+02  2.0000e+00
   9.9900e+02  0.0000e+00  1.0000e+00  1.4000e+00  9.3918e+01 -4.2700e+01
   4.9620e+00  5.2281e+03  0.0000e+00]]


### Split dataset for the active party for comparison

In [25]:
full_data_table = np.genfromtxt('dataset/bank_additional_full_filtered_balanced.csv', delimiter=',')
print(full_data_table.shape, full_data_table[0:2, :])

(7718, 21) [[ 3.9000e+01  2.0000e+00  1.0000e+00  1.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  6.0000e+00  2.0000e+00  1.2100e+02  1.0000e+00
   9.9900e+02  0.0000e+00  1.0000e+00 -1.8000e+00  9.2893e+01 -4.6200e+01
   1.3270e+00  5.0991e+03  0.0000e+00]
 [ 2.7000e+01  0.0000e+00  2.0000e+00  5.0000e+00  0.0000e+00  0.0000e+00
   1.0000e+00  0.0000e+00  3.0000e+00  3.0000e+00  5.1800e+02  2.0000e+00
   9.9900e+02  0.0000e+00  1.0000e+00  1.4000e+00  9.3918e+01 -4.2700e+01
   4.9620e+00  5.2281e+03  0.0000e+00]]


In [26]:
party_num = 2
feature_num_per_party = (bank_np_balanced.shape[1] - 1) // party_num
print("feature_num_per_party: ", feature_num_per_party)

feature_num_per_party:  10


In [27]:
bank_np_active = bank_np_balanced[:,:int(feature_num_per_party)]
print("bank_np_active.shape: ", bank_np_active.shape)
bank_np_passive = bank_np_balanced[:,int(feature_num_per_party):-1]
print("bank_np_passive.shape: ", bank_np_passive.shape)

bank_np_active.shape:  (7718, 10)
bank_np_passive.shape:  (7718, 10)


In [28]:
bank_np_label = np.reshape(bank_np_balanced[:,-1], (-1, 1))
bank_np_active = np.append(bank_np_active, bank_np_label, axis=1)
bank_np_passive = np.append(bank_np_passive, bank_np_label, axis=1)
print("bank_np_active.shape: ", bank_np_active.shape)
print("bank_np_passive.shape: ", bank_np_passive.shape)

# save the split dataset for the active and passive party
np.savetxt("dataset/bank_additional_full_filtered_balanced_active.csv", bank_np_active, delimiter=",")
np.savetxt("dataset/bank_additional_full_filtered_balanced_passive.csv", bank_np_passive, delimiter=",")

bank_np_active.shape:  (7718, 11)
bank_np_passive.shape:  (7718, 11)


In [29]:
# duplicate the label column for the server in splitnn, to use torch data loader
bank_np_label = np.append(bank_np_label, bank_np_label, axis=1)
np.savetxt("dataset/bank_additional_full_filtered_balanced_server.csv", bank_np_label, delimiter=",")
